In [20]:
!pip install -r requirements.txt

In [21]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_together import ChatTogether
from langchain_community.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
import os

In [22]:
llm = ChatTogether(
    together_api_key="API_Key",
    model="mistralai/Mistral-7B-Instruct-v0.1",
)

In [23]:
'''os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_API_KEY"]="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"
os.environ["LANGCHAIN_PROJECT"]="workshop-test"'''

'os.environ["LANGCHAIN_TRACING_V2"]="true"\nos.environ["LANGCHAIN_API_KEY"]="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"\nos.environ["LANGCHAIN_PROJECT"]="workshop-test"'

In [24]:
'''from langsmith import Client

client = Client()'''

'from langsmith import Client\n\nclient = Client()'

In [25]:
llm.invoke("hi")

AIMessage(content=' Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 9, 'total_tokens': 19, 'completion_tokens_details': None, 'prompt_tokens_details': None, 'cached_tokens': 0}, 'model_name': 'mistralai/Mistral-7B-Instruct-v0.1', 'system_fingerprint': None, 'id': 'nv9Lb83-6UHjtw-94419bb4ddf5b859', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None}, id='run--5d0170ae-8b22-4eb5-98de-1987fb4c75d4-0', usage_metadata={'input_tokens': 9, 'output_tokens': 10, 'total_tokens': 19, 'input_token_details': {}, 'output_token_details': {}})

In [26]:
(llm|StrOutputParser()).invoke("Answer in one sentence: Which countries were most represented in olymphics 2024?")

" The countries with the most representation in the Olympics 2024 would be determined by the specific games and their respective host countries' policies on athlete participation."

In [27]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://www.economicsobservatory.com/what-happened-at-the-2024-olympics")

In [28]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1200,
    chunk_overlap=100,
    separators=["\n\n", "\n", ".", ";", ",", " ", ""],
)

In [29]:
documents = loader.load()

In [30]:
docs_split = text_splitter.split_documents(documents)

In [31]:
embedding_function = SentenceTransformerEmbeddings(model_name="BAAI/bge-small-en-v1.5")

In [32]:
from langchain_chroma import Chroma

vector_store = Chroma(
    collection_name="collection-1",
    embedding_function=embedding_function,
    persist_directory="./vectordb",  # Where to save data locally, remove if not neccesary
)

In [33]:
retriever = vector_store.as_retriever()

In [34]:
retriever.add_documents(docs_split)

['488e643f-0fed-4fdd-9053-d4b4cf469097',
 '6b5fcff2-7f27-48ff-8454-5156f0404dc5',
 'c53d39a8-24fd-409a-b909-eb30e159d000',
 '6a171720-5a64-4ae7-ac61-bb71c68e2a37',
 '53d383dc-67bc-4dab-a569-6dc68f6fc5c8',
 '7109aae7-dc1e-47a5-8403-c2c2aaadfc98',
 '2d4ece13-9573-4095-90b1-43a46230ab78',
 'b42fe231-f119-4bed-a34b-ea2fdf962f74',
 'f6c4359f-24c4-492a-907e-2842185922b3',
 'f14c3192-b5b1-4fa7-93da-010404f57804',
 '4cbaa50c-94bd-49cf-ae08-0c31c13adc94',
 '28df9dd8-57ce-4b70-b9e8-e922ab6df785',
 'a1736af7-8e5e-4152-a4db-b0e217a03c1e',
 'f0c910cf-8b77-471a-9621-04072be69f56',
 'a6514d0d-4d49-4179-92f8-cbe367c54dc2',
 '454cefcf-b620-447f-81e0-64a4155725ca',
 'd21ea560-6fd2-4d9c-9e6d-cd2751e3ef7a',
 'c12ec5e8-3e80-4bce-b18e-f4bb8f551456',
 'f20505e3-d8c2-4bac-80cf-e01ca002d88a',
 '6ffe0017-59f1-417b-8bc7-13c2b9dd4001']

In [35]:
from langchain_core.prompts import ChatPromptTemplate

chat_template = ChatPromptTemplate.from_template("""You are an useful assistant.
                                                    Answer the following user query with the help of facts provided below.
                                                    Query: {query}
                                                    here are some facts that might help to answer the query:
                                                    {context}
                                                 """)

In [37]:
rag_chain = (
    {"context": retriever | format_docs, "query": RunnablePassthrough()}
    | chat_template
    | llm
    | StrOutputParser()
)


In [38]:
rag_chain.invoke("Answer in one sentence: Which countries were most represented in olymphics 2024?")

' The 2024 Olympics were hosted by Paris and featured 206 territories represented, alongside the International Olympic Committee (IOC) Refugee Olympic Team. The increase in participating countries has been consistent aside from a noticeable break in 1980 when the Moscow games saw the largest Olympic boycott in history. No single country significantly benefited from the absence of Russia in Paris, but China and the United States claimed the most medals (ten and nine respectively), while others shared the rest.'